# Perturbation registries

The pertdb module is designed to model complex <a href="../pertdb.experiment.html">pertdb.Experiment</a> of any <a href="../pertdb.experimenttype.html">pertdb.ExperimentType</a> that include perturbations in the experimental design.

Perturbations are intentional disruptions in biological systems, such as <a href="../pertdb.geneticperturbation.html">pertdb.GeneticPerturbation</a>, <a href="../pertdb.compoundperturbation.html">pertdb.CompoundPerturbation</a>, or <a href="../pertdb.environmentalperturbation.html">pertdb.EnvironmentalPerturbation</a>, to study their effects on molecular and cellular processes.
Such perturbations sometimes also have known <a href="../pertdb.perturbationtarget.html">pertdb.PerturbationTarget</a> which can be one or serveral {class}`~docs:bionty.Gene`, {class}`~docs:bionty.Pathway`, or {class}`~docs:bionty.Protein`.
While single perturbations are common, it is also possible to combine several perturbations into a <a href="../pertdb.combinationperturbation.html">pertdb.CombinationPerturbation</a>.

In [ ]:
# pip install lamindb pertdb
!lamin init --storage ./test-pert-registries --modules bionty,pertdb

In [ ]:
import bionty as bt
import pertdb

Gefitinib is primarily a pharmaceutical drug, specifically a targeted therapy used in cancer treatment, particularly for non-small cell lung cancer (NSCLC).
Let's assume that we want to better understand the effectiveness of gefitinib and EGFR/KRAS knockdown combination perturbation for people that have a smoking history.
In our early experimental setup, we therefore subject mice to smoke.

In [ ]:
bt.settings.organism = "mouse"

## Genetic perturbations

We create two <a href="../pertdb.geneticperturbation.html">pertdb.GeneticPerturbation</a> records associated with the corresponding <a href="../pertdb.perturbationtarget.html">pertdb.PerturbationTarget</a>.

In [ ]:
EGFR_kd = pertdb.GeneticPerturbation(
    type="CRISPR-Cas9",
    name="EGFR knockdown",
    sequence="AGCTGACCGTGA",
    on_target_score=85,
    off_target_score=15,
).save()

In [ ]:
EGFR_gene = bt.Gene.from_source("EGFR").save()
EGFR_kd_target = pertdb.PerturbationTarget(name="EGFR").save()
EGFR_kd_target.genes.add(EGFR_gene)

In [ ]:
EGFR_kd.targets.add(EGFR_kd_target)
EGFR_kd_target

In [ ]:
KRAS_kd = pertdb.GeneticPerturbation(
    type="CRISPR-Cas9",
    name="KRAS",
    sequence="TTGGTGGTGAACT",
    on_target_score=100,
    off_target_score=20,
).save()

In [ ]:
KRAS_gene = bt.Gene.from_source("KRAS").save()
KRAS_kd_target = pertdb.PerturbationTarget(name="KRAS").save()
KRAS_kd_target.genes.add(KRAS_gene)

In [ ]:
KRAS_kd.targets.add(KRAS_kd_target)
KRAS_kd_target

## Compound perturbations

Next, we create a <a href="../pertdb.compound.html">pertdb.Compound</a> records including associated <a href="../pertdb.perturbationtarget.html">pertdb.PerturbationTarget</a>.

In [ ]:
gefitinib = pertdb.Compound(name="gefitinib").save()

Gefitinib is a tyrosine kinase inhibitor (TKI) that specifically targets the epidermal growth factor receptor (EGFR) pathway and the EGFR protein.
Therefore, we can also define the <a href="../pertdb.perturbationtarget.html"> and associate it with a :class:`bionty.Pathway` and :class:`bionty.Protein`.

In [ ]:
egfr_pathway = bt.Pathway.from_source(
    name="epidermal growth factor receptor activity"
).save()
egfr_protein = bt.Protein.from_source(uniprotkb_id="Q5SVE7").save()
EGFR_kd_target.pathways.add(egfr_pathway)
EGFR_kd_target.proteins.add(egfr_protein)
EGFR_kd_target

In [ ]:
gefitinib.targets.add(EGFR_kd_target)
gefitinib

## Environmental perturbations

In [ ]:
smoking = pertdb.EnvironmentalPerturbation(
    name="smoking status measurement", ontology_id="EFO:0006527"
).save()

## Combination Perturbation

Now we can combine all individual perturbations into a single <a href="../pertdb.combinationperturbation.html">pertdb.CombinationPerturbation</a>.

In [ ]:
from datetime import timedelta

gefitinib_pert = pertdb.CompoundPerturbation(
    name="gefitinib 10uM treatment",
    concentration=10.0,
    concentration_unit="uM",
    duration=timedelta(hours=24),
    compound=gefitinib,
).save()

In [ ]:
combination_perturbation = pertdb.CombinationPerturbation(
    name="gefitinib and EGFR/KRAS knockdown combination perturbation subject to smoking"
).save()
combination_perturbation.genetic_perturbations.set([EGFR_kd, KRAS_kd])
combination_perturbation.compound_perturbations.add(gefitinib_pert)
combination_perturbation.environmental_perturbations.add(smoking)

In [ ]:
combination_perturbation